In [ ]:
import gzip
import struct
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import torch
rng = np.random.default_rng() 

def load_mnist_images(filename):
    with gzip.open(filename, 'rb') as f:
        # Read the header information: magic number, number of images, rows, and columns
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        if magic != 2051:
            raise ValueError(f'Invalid magic number {magic} in MNIST image file: {filename}')
        
        # Read the remaining bytes (each image is rows*cols bytes)
        buf = f.read(num_images * rows * cols)
        # Convert the byte data to a numpy array and reshape it
        images = np.frombuffer(buf, dtype=np.uint8).reshape(num_images, rows*cols)
        return images
    
def load_mnist_labels(filename):
    with gzip.open(filename, 'rb') as f:
        # Read the header information: magic number and number of labels
        magic, num_labels = struct.unpack('>II', f.read(8))
        if magic != 2049:
            raise ValueError(f'Invalid magic number {magic} in MNIST label file: {filename}')
        
        # Read the labels (each label is 1 byte)
        buf = f.read(num_labels)
        labels = np.frombuffer(buf, dtype=np.uint8)
        return labels
    
def encoder(labels,N_digits):
    codebook = np.zeros((labels.shape[0],10))
    for i in range(labels.shape[0]):
        codebook[i,labels[i]] = 1 
    return codebook

In [ ]:
train_images = load_mnist_images('train-images-idx3-ubyte.gz')
train_labels = load_mnist_labels('train-labels-idx1-ubyte.gz')

test_images = load_mnist_images('t10k-images-idx3-ubyte.gz')
test_labels = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

# Normalize the images to a [0,1] range

train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

dimension = train_images.shape[1]

In [ ]:
training_data = np.ones((train_images.shape[0],dimension+1))
training_data[:,0:dimension] = train_images

test_data = np.ones((test_images.shape[0],dimension+1))
test_data[:,0:dimension] = test_images

In [ ]:
training_tensor = torch.from_numpy(training_data)
test_tensor = torch.from_numpy(test_data)
T = encoder(train_labels,10)
T_test = encoder(test_labels,10)
T_tensor = torch.from_numpy(T)
T_test_tensor = torch.from_numpy(T_test)

In [ ]:
def softmax(z):
    return torch.exp(z)/(torch.sum(torch.exp(z),dim = 0))
def sigmoid(z):
    return (1/(1+torch.exp(-z)))
def d_sigmoid(z):
    return (1-sigmoid(z))*sigmoid(z)
    
def ReLU(z):
    return torch.relu(z)

def d_ReLU(z):
    return torch.where(z > 0, torch.tensor(1.0), torch.tensor(0.0))

def feedforward(W,X,**kwargs):
    layer = kwargs.get("layer")
    if layer == 1:
        return sigmoid(W @ X)
    elif layer == 2:
        return softmax(W @ X)

def backpropagation(T, Y, X, **kwargs):
    W1 = kwargs.get("W1")  
    W2 = kwargs.get("W2")  
    layer = kwargs.get("layer")

    if layer == 2:
        A = T - Y  # shape: [n_out, batch_size]

        A_batch = A.t()    # shape: [batch_size, n_out]
        X_batch = X.t()    # here X is the hidden activation; shape: [batch_size, n_hidden]
        dW = - torch.bmm( A_batch.unsqueeze(2),  # shape: [batch_size, n_out, 1]
                           X_batch.unsqueeze(1))  # shape: [batch_size, 1, n_hidden]
        dW = dW.sum(dim=0)  # resulting shape: [n_out, n_hidden]
        return dW

    elif layer == 1:
        neuron_coeff = - (W2.t() @ (T - Y))  # shape: [n_hidden, batch_size]
        hidden_deriv = d_sigmoid(W1 @ X)  # shape: [n_hidden, batch_size]

        delta_hidden = neuron_coeff * hidden_deriv  # shape: [n_hidden, batch_size]
        delta_batch = delta_hidden.t()  # shape: [batch_size, n_hidden]
        X_batch     = X.t()             # shape: [batch_size, input_dim]

        dW = torch.bmm(delta_batch.unsqueeze(2),  # shape: [batch_size, n_hidden, 1]
                         X_batch.unsqueeze(1))      # shape: [batch_size, 1, input_dim]
        dW = dW.sum(dim=0)  # resulting shape: [n_hidden, input_dim]
        return dW
def binarize_max_per_row(A):
    max_per_row, _ = torch.max(A, dim=1, keepdim=True)  
    max_elements = (A == max_per_row).to(torch.int)  
    return max_elements


In [ ]:
H = 100
W1 = torch.normal(0,1,(H,dimension+1),dtype = torch.float64)
W2 = torch.normal(0,1,(10,H),dtype = torch.float64)

training_tensor = training_tensor.to('cuda')
test_tensor = test_tensor.to('cuda')
W1 = W1.to('cuda')
W2 = W2.to('cuda')
T_tensor = T_tensor.to('cuda')
T_test_tensor = T_test_tensor.to('cuda')

In [ ]:
batch_size = 500
R = int(training_data.shape[0]/batch_size)
learning_rate = 10**(-2)
N = 2500

test_Errors = np.zeros(N)
training_Errors = np.zeros(N)

for i in tqdm(range(N), desc="Training Epochs"):
    V = feedforward(W1,training_tensor[(i%R)*batch_size:((i%R)+1)*batch_size,:].t(),layer=1)
    Y = feedforward(W2,V,layer=2)
    dW2 = backpropagation(T_tensor[(i%R)*batch_size:((i%R)+1)*batch_size,:].t(),Y,V,layer=2)
    dW1 = backpropagation(T_tensor[(i%R)*batch_size:((i%R)+1)*batch_size,:].t(),Y,training_tensor[(i%R)*batch_size:((i%R)+1)*batch_size,:].t(),layer=1, W1 = W1, W2 = W2)
    W1 = W1 - learning_rate*dW1
    W2 = W2 - learning_rate*dW2

    predictions = feedforward(W2,feedforward(W1,training_tensor.t(),layer=1),layer =2)
    encoded_predictions = binarize_max_per_row(predictions.t())
    errors = torch.abs(encoded_predictions-T_tensor)
    errors = torch.sum(errors, dim = 1)
    errors = torch.where(errors > 0, torch.tensor(1.0), torch.tensor(0.0))
    errors = torch.sum(errors)/errors.shape[0]

    predictions_test = feedforward(W2,feedforward(W1,test_tensor.t(),layer=1),layer =2)
    encoded_predictions_test = binarize_max_per_row(predictions_test.t())
    errors_test = torch.abs(encoded_predictions_test-T_test_tensor)
    errors_test = torch.sum(errors_test, dim = 1)
    errors_test = torch.where(errors_test > 0, torch.tensor(1.0), torch.tensor(0.0))
    errors_test = torch.sum(errors_test)/errors_test.shape[0]
        
    training_Errors[i] = errors.detach().cpu().numpy()
    test_Errors[i] = errors_test.detach().cpu().numpy()
    print(errors.detach().cpu().numpy())

torch.cuda.empty_cache()

In [ ]:
np.savetxt('trainingErrorsNNtwolayerSGDH50.txt', training_Errors, fmt='%.4f', delimiter=',')
np.savetxt('testErrorsNNtwolayerSGDH50.txt', test_Errors, fmt='%.4f', delimiter=',')

